In [1]:
## This code is inteded to create a sales_date that is a proper date and 
#  create two dummies that show the competition and promo2 for each store day by day.
#  EO 28.10.2021

In [2]:
##TODO: Check that the two dummies are correct and delete 
##TODO:
   
# 15  Promo2SinceWeek            314828 non-null  float64       
# 16  Promo2SinceYear            314828 non-null  float64       
# 17  PromoInterval              314828 non-null  object        
# 18  temp_date                  422043 non-null  float64       
# 19  Promo2_start_date          314828 non-null  datetime64[ns]
# 22  Competition_start_date     422043 non-null  datetime64[ns]


In [13]:
import pandas as pd
import numpy as np

import pandas_profiling as pp
from pandas_profiling import ProfileReport


# reading two csv files
data1 = pd.read_csv('data/train.csv', dtype={'Store':'string', 'StateHoliday':'string'}) # removes the error message
data2 = pd.read_csv('data/store.csv', dtype={'Store':'string', 'StoreType':'string', 'Assortment':'string'})

# using merge function by setting how='inner'
total = pd.merge(data1, data2,
                   on='Store',
                   how='inner')
print(data1.info())

print(data2.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637774 entries, 0 to 637773
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           637774 non-null  object 
 1   Store          618473 non-null  string 
 2   DayOfWeek      618757 non-null  float64
 3   Sales          618747 non-null  float64
 4   Customers      618683 non-null  float64
 5   Open           618588 non-null  float64
 6   Promo          618580 non-null  float64
 7   StateHoliday   618520 non-null  string 
 8   SchoolHoliday  618437 non-null  float64
dtypes: float64(6), object(1), string(2)
memory usage: 43.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   string 
 1   StoreType                  1115 non-null   str

In [19]:
from datetime import datetime 
from datetime import date 

## Create a new column that is a real date.
## For a time series we need real dates.
total['sales_date'] = pd.to_datetime(total['Date'])
total.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 618473 entries, 0 to 618472
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Date                       618473 non-null  object        
 1   Store                      618473 non-null  string        
 2   DayOfWeek                  600036 non-null  float64       
 3   Sales                      600028 non-null  float64       
 4   Customers                  599957 non-null  float64       
 5   Open                       599832 non-null  float64       
 6   Promo                      599791 non-null  float64       
 7   StateHoliday               599873 non-null  string        
 8   SchoolHoliday              599686 non-null  float64       
 9   StoreType                  618473 non-null  string        
 10  Assortment                 618473 non-null  string        
 11  CompetitionDistance        616838 non-null  float64 

In [20]:
def create_a_timeseries_dummy_from_weeks(df, var_week, var_year, date_new, var_dummy):
    #Create a date out of the Promo2 variables so that we can create a time-series variable for Promo2
    df['temp_date'] = df[var_year] * 1000 + df[var_week]  * 10 + 0
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%W%w')
    ## Create an array to Group the data by group. and make dummy Store by Store
    for Store, grouped in df.groupby('Store'):
      if [df['sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])
    df.head()

    
def create_a_timeseries_dummy_from_months(df, var_month, var_year, date_new, var_dummy):
    #Create a date out of the Competition variables so that we can create a time-series variable for Competitioin
    df['temp_date'] = df[var_year] * 1000 + df[var_month]  * 10 + 0
    ## Create an array to Group the data by group. and make dummy Store by Store
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%m%w')
    for Store, grouped in df.groupby('Store'):
      if [df['sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])    
    df.head()

In [21]:
total.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,temp_date,Promo2_start_date,sales_date
0,2013-01-01,1115,2.0,0.0,0.0,0.0,0.0,a,1.0,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012220.0,2012-06-03,2013-01-01
1,2013-01-02,1115,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012220.0,2012-06-03,2013-01-02
2,2013-01-03,1115,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012220.0,2012-06-03,2013-01-03
3,2013-01-04,1115,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012220.0,2012-06-03,2013-01-04
4,2013-01-05,1115,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,...,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2012220.0,2012-06-03,2013-01-05


In [22]:
create_a_timeseries_dummy_from_weeks(df=total         ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

In [23]:
create_a_timeseries_dummy_from_months(df=total                   ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )


In [24]:
total.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,temp_date,Promo2_start_date,sales_date,Promo2_yes,Competition_start_date,Competition_yes
0,2013-01-01,1115,2.0,0.0,0.0,0.0,0.0,a,1.0,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2012-06-03,2013-01-01,1,NaT,1
1,2013-01-02,1115,3.0,3697.0,305.0,1.0,0.0,0,1.0,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2012-06-03,2013-01-02,1,NaT,1
2,2013-01-03,1115,4.0,4297.0,300.0,1.0,0.0,0,1.0,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2012-06-03,2013-01-03,1,NaT,1
3,2013-01-04,1115,5.0,4540.0,326.0,1.0,0.0,0,1.0,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2012-06-03,2013-01-04,1,NaT,1
4,2013-01-05,1115,6.0,4771.0,339.0,1.0,0.0,0,1.0,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2012-06-03,2013-01-05,1,NaT,1


In [25]:
total.Promo2.value_counts()
total.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 618473 entries, 0 to 618472
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Date                       618473 non-null  object        
 1   Store                      618473 non-null  string        
 2   DayOfWeek                  600036 non-null  float64       
 3   Sales                      600028 non-null  float64       
 4   Customers                  599957 non-null  float64       
 5   Open                       599832 non-null  float64       
 6   Promo                      599791 non-null  float64       
 7   StateHoliday               599873 non-null  string        
 8   SchoolHoliday              599686 non-null  float64       
 9   StoreType                  618473 non-null  string        
 10  Assortment                 618473 non-null  string        
 11  CompetitionDistance        616838 non-null  float64 

In [45]:
from datetime import datetime
f = labda x: x.strftime('%A')
total.apply(f, axis='columns')

#total['weekday'] = total['sales_date'].strftime('%A')

SyntaxError: invalid syntax (3205939962.py, line 2)

In [ ]:
total.head()